In [3]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
import faiss

# Загрузка модели и токенизатора BERT
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Загрузка данных из CSV
df = pd.read_csv('final_data.csv')

# Максимальная длина текста
MAX_LEN = 300

# Функция для встраивания текста с использованием BERT
def embed_bert_cls(text, model=model, tokenizer=tokenizer):
    t = tokenizer(text,
                  padding=True,
                  truncation=True,
                  return_tensors='pt',
                  max_length=MAX_LEN)
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().squeeze()

# Загрузка предварительно вычисленных векторов
embeddings = np.loadtxt('embeddings.txt')
embeddings_tensor = [torch.tensor(embedding) for embedding in embeddings]

# Создание индекса Faiss
embeddings_matrix = np.stack(embeddings)
index = faiss.IndexFlatIP(embeddings_matrix.shape[1])
index.add(embeddings_matrix)

# Текст запроса
text = 'добрую сказку с плохим концом для детей'

# Встраивание запроса и поиск ближайших векторов с использованием Faiss
query_embedding = embed_bert_cls(text)
query_embedding = query_embedding.numpy().astype('float32')
k, indices = index.search(np.expand_dims(query_embedding, axis=0), 3)

# Вывод результатов
for i in indices[0]:
    print(df['title'][i])

ДХЛ. Красная шапочка и другие сказки
Ослиная шкура
Рождественское чудо мистера Туми
